In [1]:
import xml.etree.ElementTree as ET
import os
from os import getcwd
from os.path import join
import glob
import shutil
from tqdm import tqdm

In [9]:
# classes = ['c','1','2','3','4']  # 标注时的标签
classes = ["wjj", "ztc", "ttc",'jzlaji']
data_dir='D:/temp_dataset/puzhou_ztc/'
xml = 'zhatuche_xml'
txt = 'zhatuche_txt2'

xml_path = os.path.join(data_dir, xml)
txt_path = os.path.join(data_dir, txt)
if not os.path.exists(txt_path): #判断所在目录下是否有该文件名的文件夹
    os.mkdir(txt_path) 
xml_files = os.listdir(xml_path)
print(len(xml_files))


869


In [10]:
'''
xml中框的左上角坐标和右下角坐标(x1,y1,x2,y2)
》》txt中的中心点坐标和宽和高(x,y,w,h)，并且归一化
'''
def convert(size, box):
    dw = 1. / size[0]
    dh = 1. / size[1]
    x = (box[0] + box[1]) / 2.0
    y = (box[2] + box[3]) / 2.0
    w = box[1] - box[0]
    h = box[3] - box[2]
    x = x * dw
    w = w * dw
    y = y * dh
    h = h * dh
    return (x, y, w, h)
    
def convert_annotation(xml_path,txt_path,file_name,fname):
    in_file = open(os.path.join(xml_path, file_name), encoding='utf-8') #读取xml
    out_file = open(os.path.join(txt_path, fname+'.txt'), 'w') #保存txt
    
    tree = ET.parse(in_file)
    root = tree.getroot()
    size = root.find('size')
    w = int(size.find('width').text)
    h = int(size.find('height').text)
    for obj in root.iter('object'):
        # difficult = obj.find('difficult').text
        cls = obj.find('name').text
        # if cls not in classes or int(difficult) == 1:
        if cls not in classes:
            continue
        cls_id = classes.index(cls)#获取类别索引
        xmlbox = obj.find('bndbox')
        b = (float(xmlbox.find('xmin').text), float(xmlbox.find('xmax').text), float(xmlbox.find('ymin').text),
             float(xmlbox.find('ymax').text))
        bb = convert((w, h), b)
        out_file.write(str(cls_id) + " " + " ".join([str('%.6f'%a) for a in bb]) + '\n')
print(len(xml_files),'个文件')  


for file_name in tqdm(xml_files):
    fname = file_name.split('.xml')[0]
    convert_annotation(xml_path,txt_path,file_name,fname)
print("Over!!!")  


869 个文件


100%|██████████████████████████████████████████████████████████████████████████████| 869/869 [00:00<00:00, 1907.58it/s]

Over!!!


In [11]:
# 删除空文本文件
for filename in tqdm(os.listdir(txt_path)):
    # 判断是否为文本文件
    if filename.endswith(".txt"):
        # 判断文件是否为空
        if os.path.getsize(os.path.join(txt_path, filename)) == 0:
            # 删除文件
            os.remove(os.path.join(txt_path, filename))

100%|█████████████████████████████████████████████████████████████████████████████| 869/869 [00:00<00:00, 13317.49it/s]
